# Quantum Approximate Membership (QAM) Experiments

Interactive notebook for exploring QAM performance and comparing with classical Bloom filters.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

from sim.qam import QAM, create_noise_model
from sim.utils import make_hash_functions

# Set random seed for reproducibility
np.random.seed(42)

print("QAM module loaded successfully!")

## 1. Basic QAM Usage

In [ ]:
# Initialize QAM
m = 16  # number of qubits
k = 3   # number of hash functions
theta = np.pi / 4

qam = QAM(m=m, k=k, theta=theta)

# Create test items
items = [b"apple", b"banana", b"cherry"]
query_items = [b"apple", b"grape"]  # First is in set, second is not

print(f"QAM initialized: m={m}, k={k}, θ={theta:.3f}")
print(f"Inserted items: {items}")
print(f"Query items: {query_items}")

In [ ]:
# Query using statevector (exact, no noise)
for q_item in query_items:
    prob = qam.query_statevector(items, q_item)
    print(f"Query '{q_item.decode()}': probability={prob:.4f}")

In [ ]:
# Query with shots (realistic simulation)
shots = 512
for q_item in query_items:
    exp = qam.query(items, q_item, shots=shots)
    print(f"Query '{q_item.decode()}': expectation={exp:.4f} (shots={shots})")

## 2. Visualize Circuit

In [ ]:
# Build and visualize query circuit
qc = qam.build_query_circuit(items, b"apple")
print(f"Circuit depth: {qc.depth()}")
print(f"Circuit width: {qc.num_qubits}")

# Draw circuit (first 8 qubits for clarity)
qc.draw('mpl', scale=0.8)

## 3. Accuracy vs Shots

In [ ]:
# Test how accuracy changes with shot count
shot_values = [64, 128, 256, 512, 1024, 2048]
n_trials = 10

# Generate test data
def generate_items(n):
    return [bytes(np.random.randint(0, 256, 8)) for _ in range(n)]

inserted_items = generate_items(20)
positive_tests = inserted_items[:5]
negative_tests = generate_items(5)

results = {'shots': [], 'fp_rate': [], 'fn_rate': []}

for shots in shot_values:
    fp_rates = []
    fn_rates = []
    
    for _ in range(n_trials):
        # False positives
        fps = sum(1 for item in negative_tests if qam.query(inserted_items, item, shots=shots) >= 0.5)
        fp_rates.append(fps / len(negative_tests))
        
        # False negatives
        fns = sum(1 for item in positive_tests if qam.query(inserted_items, item, shots=shots) < 0.5)
        fn_rates.append(fns / len(positive_tests))
    
    results['shots'].append(shots)
    results['fp_rate'].append(np.mean(fp_rates))
    results['fn_rate'].append(np.mean(fn_rates))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(results['shots'], results['fp_rate'], marker='o', label='False Positive Rate', linewidth=2)
ax.plot(results['shots'], results['fn_rate'], marker='s', label='False Negative Rate', linewidth=2)
ax.set_xlabel('Number of Shots', fontsize=12)
ax.set_ylabel('Error Rate', fontsize=12)
ax.set_title('QAM Accuracy vs Shots', fontsize=14, fontweight='bold')
ax.set_xscale('log', base=2)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nOptimal shots for low FP: {results['shots'][np.argmin(results['fp_rate'])]}")

## 4. Robustness to Noise

In [ ]:
# Test with different noise levels
noise_levels = [0.0, 0.001, 0.005, 0.01, 0.02]
shots = 512

noise_results = {'noise': [], 'fp_rate': []}

for noise in noise_levels:
    noise_model = create_noise_model(noise) if noise > 0 else None
    
    fps = sum(1 for item in negative_tests if qam.query(inserted_items, item, shots=shots, noise_model=noise_model) >= 0.5)
    fp_rate = fps / len(negative_tests)
    
    noise_results['noise'].append(noise)
    noise_results['fp_rate'].append(fp_rate)
    print(f"Noise ε={noise:.3f}: FP rate={fp_rate:.3f}")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(noise_results['noise'], noise_results['fp_rate'], marker='o', linewidth=2, color='#A23B72')
ax.set_xlabel('Noise Rate (ε)', fontsize=12)
ax.set_ylabel('False Positive Rate', fontsize=12)
ax.set_title('QAM Robustness vs Noise', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. QAM vs Classical Bloom Filter

In [ ]:
def classical_bloom_filter(items, query_item, m, k):
    """Classical Bloom filter baseline."""
    bit_array = [False] * m
    hash_fns = make_hash_functions(k)
    
    # Insert
    for item in items:
        item_hash = hash(item)
        for h in hash_fns:
            bit_array[h(item_hash) % m] = True
    
    # Query
    query_hash = hash(query_item)
    for h in hash_fns:
        if not bit_array[h(query_hash) % m]:
            return False
    return True

# Compare at different memory sizes
m_values = [8, 16, 32, 64]
comparison = {'m': [], 'qam_fp': [], 'classical_fp': []}

for m in m_values:
    qam_temp = QAM(m=m, k=3, theta=np.pi/4)
    
    # QAM false positives
    qam_fps = sum(1 for item in negative_tests if qam_temp.query(inserted_items, item, shots=512) >= 0.5)
    qam_fp_rate = qam_fps / len(negative_tests)
    
    # Classical false positives
    classical_fps = sum(1 for item in negative_tests if classical_bloom_filter(inserted_items, item, m, 3))
    classical_fp_rate = classical_fps / len(negative_tests)
    
    comparison['m'].append(m)
    comparison['qam_fp'].append(qam_fp_rate)
    comparison['classical_fp'].append(classical_fp_rate)
    
    print(f"m={m:2d}: QAM FP={qam_fp_rate:.3f}, Classical FP={classical_fp_rate:.3f}")

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(comparison['m'], comparison['qam_fp'], marker='o', label='QAM', linewidth=2)
ax.plot(comparison['m'], comparison['classical_fp'], marker='s', label='Classical Bloom', linewidth=2)
ax.set_xlabel('Memory (qubits/bits)', fontsize=12)
ax.set_ylabel('False Positive Rate', fontsize=12)
ax.set_title('QAM vs Classical Bloom Filter', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Load Factor Analysis

In [ ]:
# Test different set sizes (load factors)
m = 32
k = 3
set_sizes = [8, 16, 24, 32, 40]

load_results = {'load_factor': [], 'qam_fp': [], 'classical_fp': []}

qam_temp = QAM(m=m, k=k, theta=np.pi/4)

for set_size in set_sizes:
    items = generate_items(set_size)
    test_items = generate_items(10)
    
    qam_fps = sum(1 for item in test_items if qam_temp.query(items, item, shots=512) >= 0.5)
    classical_fps = sum(1 for item in test_items if classical_bloom_filter(items, item, m, k))
    
    load_factor = set_size / m
    load_results['load_factor'].append(load_factor)
    load_results['qam_fp'].append(qam_fps / len(test_items))
    load_results['classical_fp'].append(classical_fps / len(test_items))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(load_results['load_factor'], load_results['qam_fp'], marker='o', label='QAM', linewidth=2)
ax.plot(load_results['load_factor'], load_results['classical_fp'], marker='s', label='Classical', linewidth=2)
ax.set_xlabel('Load Factor (|S|/m)', fontsize=12)
ax.set_ylabel('False Positive Rate', fontsize=12)
ax.set_title('Accuracy vs Load Factor', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey observations:")
print(f"- QAM performs best at load factors < 0.5")
print(f"- Both QAM and classical degrade at high load factors")

## Next Steps

1. Run full parameter sweep: `python experiments/sweeps.py --sweep`
2. Generate publication plots: `python experiments/plotting.py --results results/qam_sweep_*.csv`
3. Explore theoretical bounds in `theory/` folder